In [2]:
import pandas as pd
from datetime import datetime

In [3]:
flights = pd.read_parquet('data/flight_data.parquet')
weather = pd.read_parquet('weather.parquet')



In [4]:
# First, let's drop the rows with null values in Origin or DepTime from flights
flights = flights.dropna(subset=['Origin', 'DepTime'])

# Convert both category columns to the same dtype
flights["Origin"] = flights["Origin"].astype(str)
weather['IATA'] = weather['IATA'].astype(str)

# Create the datetime columns
mins = pd.to_timedelta(flights["DepTime"] // 100 * 60 + flights["DepTime"] % 100, unit='m')
flights["dep_datetime"] = pd.to_datetime(flights["FlightDate"]) + mins
weather["date"] = pd.to_datetime(weather["date"].apply(lambda x: x.replace(tzinfo=None)))

# Sort the DataFrames
flights.sort_values(by=["dep_datetime"], inplace=True)
weather.sort_values(by=["date"], inplace=True)

# Perform the merge
merged = pd.merge_asof(flights, weather,
                      left_by='Origin', left_on='dep_datetime',
                      right_by='IATA', right_on='date', direction='nearest')

# Drop the redundant columns
merged.drop(columns=['FlightDate', 'IATA', 'date'], inplace=True)

In [5]:
merged

,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,...,wind_direction_10m,wind_direction_80m,wind_direction_120m,wind_direction_180m,wind_gusts_10m,temperature_80m,temperature_120m,temperature_180m,latitude,longitude
0,2023,1,1,1,7,UA,UA,19977,UA,2327.0,...,296.564970,321.215600,332.959560,292.52060,30.599998,11.7575,11.607500,9.430000,37.618999,-122.375000
1,2023,1,1,1,7,UA,UA,19977,UA,191.0,...,255.963730,291.801480,300.963680,315.00010,8.640000,23.8770,23.277000,22.477001,21.320620,-157.924228
2,2023,1,1,1,7,DL,DL,19790,DL,726.0,...,216.869980,252.525130,258.055850,260.70676,15.840000,13.1155,13.315499,12.322000,33.636700,-84.428101
3,2023,1,1,1,7,UA,UA,19977,UA,1450.0,...,168.690110,229.969800,269.076000,265.38016,9.000000,10.6945,10.644500,11.097500,40.692501,-74.168701
4,2023,1,1,1,7,UA,UA,19977,UA,2277.0,...,194.676440,195.255170,181.824050,181.56220,45.000000,13.6335,13.483500,12.623500,33.942501,-118.407997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14461781,2024,4,12,31,2,UA,UA,19977,UA,2379.0,...,305.537750,322.124940,255.963730,105.52418,7.920000,7.0680,6.968000,7.138500,47.449001,-122.308998
14461782,2024,4,12,31,2,UA,UA,19977,UA,1385.0,...,77.471200,90.000000,292.619900,315.00010,3.240000,13.4575,13.257500,11.330000,37.618999,-122.375000
14461783,2024,4,12,31,2,AA,AA,19805,AA,2792.0,...,50.194473,51.709793,56.309914,86.82024,16.560000,14.3955,14.195499,12.349500,36.080101,-115.152000
14461784,2024,4,12,31,2,AA,AA,19805,AA,2891.0,...,8.130019,7.124930,333.435030,315.00010,11.879999,13.4565,13.256500,11.974000,38.695400,-121.591003


In [6]:
# Save the merged DataFrame

merged.to_parquet('data/merged_data.parquet')